In [1]:
import numpy as np
import pandas as pd
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
from fake_useragent import UserAgent
from collections import defaultdict

Функция для сбора ссылок на авто

In [2]:
def get_page(p):
    car_links = []
    try:   
        url = 'https://auto.ru/odintsovo/cars/mercedes/all/?page={}'.format(p)
        headers = {'User-Agent': UserAgent().chrome}
        page = requests.get(url, headers=headers)
        tree = BeautifulSoup(page.content, 'html.parser')
        list_cars = tree.find_all('div', {'class' : 'ListingItem-module__container yfnNqZQFB37ZapTJ4ZqHrh9_jpc4V8kiq_ozCcSBi6A'})
        for link in range(len(list_cars)):
            one_car = list_cars[link].find('div', {'class': 'ListingItem-module__description'})
            one_car = one_car.a.get('href')
            car_links.append(one_car)
    except:
        print(response)
    return car_links

Непосредственно сбор сылок на авто с помощью цикла

In [ ]:
all_links = []
for pages in tqdm(range(1 ,101)):
    time.sleep(3)
    all_links.extend(get_page(pages))

Функция для сбора данных по каждой машине

In [3]:
lost_links = []
def getData(car_link):
    try:
    
        #car_link = 'https://auto.ru/cars/used/sale/mercedes/e_klasse_amg/1103623785-4d11f3da/'
        response = requests.get(car_link, headers = {'User-Agent':UserAgent().chrome}, timeout = 15)
        tree = BeautifulSoup(response.content, 'html.parser')
        #Заголовок
        head = tree.find('div', {'class': 'LayoutSidebar__content'})
        name = head.find('div', {'class': 'CardSidebarActions'})
        #Название авто
        name = head.find('div', {'class': 'CardSidebarActions__title'}).text
        #Цена
        price = head.find('div', {'class': 'CardSidebarActions__price-caption'})
        price = price.find('span', {'class': 'OfferPriceCaption__price'}).text[:-2]
        price = price.replace(u'\xa0', u'')
        price# = int(price)

        #Характеристики
        characteristics = head.find('div', {'class': 'CardOfferBody__twoColumns'})
        #Год выпуска
        year = characteristics.find('li', {'class': 'CardInfoRow CardInfoRow_year'})
        year = "" if not year else year.a.text
        #Пробег
        kilometres = characteristics.find('li', {'class': 'CardInfoRow CardInfoRow_kmAge'})
        kilometres = kilometres.find_all('span', {'class': 'CardInfoRow__cell'})
        kilometres = "" if not kilometres else kilometres[1].get_text()[:-3]
        kilometres = kilometres.replace(u'\xa0', u'')
        kilometres  = int(kilometres)
        #Кузов
        body = characteristics.find('li', {'class': 'CardInfoRow CardInfoRow_bodytype'})
        body = "" if not year else body.a.text
        #Цвет
        color = characteristics.find('li', {'class': 'CardInfoRow CardInfoRow_color'})
        color = color.find_all('span', {'class': 'CardInfoRow__cell'})
        color = "" if not color else color[1].get_text()
        #Двигатель:
        engine = characteristics.find('li', {'class': 'CardInfoRow CardInfoRow_engine'})
        engine = engine.div.text.split('/')
        #Объем двигателя
        engine_volume = "" if not engine else float(engine[0][:-3])
        #Мощность двигателя
        engine_power = "" if not engine else int(engine[1].split()[0])
        #Тип двигателя
        engine_type = "" if not engine else engine[-1].split()[0]
        #Налог
        tax = characteristics.find('li', {'class': 'CardInfoRow CardInfoRow_transportTax'})
        tax = tax.find_all('span', {'class': 'CardInfoRow__cell'})
        tax = "" if not tax else int(tax[1].text.replace(u'\xa0', u'').split('/')[0][:-2])
        #Коробка передач
        gears = characteristics.find('li', {'class': 'CardInfoRow CardInfoRow_transmission'})
        gears = gears.find_all('span', {'class': 'CardInfoRow__cell'})
        gears = "" if not gears else gears[1].get_text()
        #Привод
        actuator = characteristics.find('li', {'class': 'CardInfoRow CardInfoRow_drive'})
        actuator = actuator.find_all('span', {'class': 'CardInfoRow__cell'})
        actuator = "" if not actuator else actuator[1].get_text()
        #Руль
        wheel = characteristics.find('li', {'class': 'CardInfoRow CardInfoRow_wheel'})
        wheel  = wheel.find_all('span', {'class': 'CardInfoRow__cell'})
        wheel  = "" if not wheel  else wheel[1].get_text()
        #Состояние
        condition = characteristics.find('li', {'class': 'CardInfoRow CardInfoRow_state'})
        condition  = condition.find_all('span', {'class': 'CardInfoRow__cell'})
        condition  = "" if not condition  else condition[1].get_text()
        #Владельцы
        owners = characteristics.find('li', {'class': 'CardInfoRow CardInfoRow_ownersCount'})
        owners  = owners.find_all('span', {'class': 'CardInfoRow__cell'})
        owners  = "" if not owners  else owners[1].get_text().replace(u'\xa0', u' ')
        #Сумма по кредиту
        try:
            credit = int(characteristics.find_all('div', {'class': 'CreditCalculatorDesktop__field-value'})[1].text.split('/')[0].replace(u'\xa0' , u'')[:-2])
            credit = "" if not credit else credit
        except:
            credit = '-'
        #Создадим первый словарь для будущей таблицы с описанием количества параметров

        first_table = defaultdict()
        columns = ['Комфорт', 'Безопасность', 'Мультимедиа', 'Салон', 'Обзор', 'Защита от угона', 'Элементы экстерьера']
        try:
            all_features = characteristics.find_all('div', {'class': 'AccordionSection ComplectationGroups__group'})

            #Создаем словарь с количеством описательных признаков
            for item in all_features:
                try:
                    feature = item.find('div', {'class': 'ComplectationGroups__item'})
                except:
                    first_table['wrong_findings'] = '-'
                try:
                    feature_name  = feature.find('span', {'class': 'ComplectationGroups__itemName'}).text
                    if feature_name in columns:
                        feature_quantity = int(feature.find('span', {'class': 'ComplectationGroups__itemCount'}).text)
                        first_table[feature_name] = feature_quantity
                except:
                    first_table[feature_table] = '-'
        except:
            for i in columns:
                first_table[i] = '-'

        #Создадим финальный словарь
        data_row = {"Название авто":name, "Цена":price, 
                         "Год выпуска":year, 
                        "Пробег":kilometres,
                        "Кузов":body,"Цвет":color, "Объем двигателя":engine_volume, 
                        "Мощность двигателя":engine_power, "Тип двигателя":engine_type, "Налог":tax, "Коробка передач":gears,
                        "Привод":actuator, "Руль":wheel, "Состояние":condition,"Владельцы":owners,"Сумма по кредиту":credit}
        first_table.update(data_row)
        return first_table
    except:
        pass
        lost_links.append(car_link)
        return 'no'   

Непосредственный сбор данных по каждому авто: характеристики собираются, стакаются в табличку ,и обновленная табличка сразу же сохраняется в файл.

In [ ]:
for car_link in tqdm(links_final):
    time.sleep(3)
    data = getData(car_link)
    if data == None:
        print(data)
    else:
        final_df = final_df.append(data, ignore_index=True)
        final_df.to_csv('FINALCAR_DATA', 'w')

В конце считываем данные из файла, получаяя итоговую таблицу со данными.

In [ ]:
df = pd.read_csv('FINALCAR_DATA.csv', delimiter='w')
df